In [69]:
import sys
sys.path.append('..')

## Custom Actions
from Tools.FileSystem import FileSystemAction
from Tools.SearchAction import SearchAction
from Tools.GoogleMapAction import GoogleMapAction
from Tools.NotionAction import NotionAction

import langfun as lf
import pyglove as pg
from langfun.core.agentic import action, Session
from langfun.core.modalities import Mime 


from IPython.display import display, JSON, HTML, Image, Markdown

import pandas as pd
import numpy as np

import asyncio
import os
from datetime import datetime
from typing import Any, Dict, List, Optional

youtube_key = os.environ.get('YOUTUBE_KEY')
claude_key = os.environ.get('CLAUDE_API_KEY')
gemini_key = os.environ.get('GEMINI_API_KEY')
openai_key = os.environ.get('OPENAI_API_KEY')
deepseek_key = os.environ.get('DEEPSEEK_API_KEY')
notion_key = os.environ.get('NOTION_API_TOKEN')
notion_travel_database_id = os.environ.get('NOTION_TRAVEL_DATABASE_ID')
notion_communication_database_id = os.environ.get('NOTION_COMMUNICATION_DATABASE_ID')

google_search_api_key = os.environ.get('GOOGLE_API_KEY')
google_search_cx_id = os.environ.get('GOOGLE_CX_ID')
google_maps_api_key = os.environ.get('GOOGLE_MAPS_API_KEY')

## build the LLM crew
lm_claude = lf.llms.Claude37Sonnet_20250219(api_key=claude_key, temperature=0.0)
lm_openai = lf.llms.Gpt4o_20241120(api_key=openai_key, max_tokens=16384)
lm_deepseek = lf.llms.DeepSeekR1(api_key=deepseek_key)
lm_gemini = lf.llms.Gemini2ProExp_20250205(api_key=gemini_key)


## **Requirements**

This section is that defining the requirements

In [71]:
family_interests = f"""
    We are a family of 3, two adults and one 11yrs old boy. Our family are curiosity driven, learning all different knowledge,behind the scenes, has passion to explore new things. Our family like diverse experience and balance between sensory overload and slow absorption.
    
    Here are the details:
    - Wife: Love history, art, culture, museum, archtecture, landscapes, music, performance/show, city walk, shopping, local life, hiking, running.
    - Husband: Love history, culture,science, food, adventure, hiking, running, museum, archtecture, landscapes, music, performance/show, city walk, local life. 
    - Son (11yrs): Love Millitary, science history/museum, great food, adventure, ski, snorkeling, hiking, landmarks, performance/show, shopping, adventure, local life.

    """
destination_country = "Netherlands and Belgium"
from_dates = "2025-03-30"
to_dates = "2025-04-06"
special_requirements = "Our landing and departure city will be Amsterdam, please make sure the itinerary logically flows from day to day. We will stay in Leiden for 3 days, Brughe for 1 day, and then 3 days in Delft. On 03/31, we will have a day trip to Amsterdam with pre-booked VanGogh Museum at 1pm and Rijksmuseum at 3:30PM.  We also flexibile to visit other cities that are close to these cities. "

In [72]:
class FamilyRequests(pg.Object):
    """This is the request from the family, please make sure the trip plan matches the family's interests and preferences, logically flows from day to day, and consider the special requirements"""

    family_interests: str
    destination_country: str
    from_dates: str
    to_dates: str
    special_requirements: str

family_requests = FamilyRequests(
    family_interests=family_interests,
    destination_country=destination_country,
    from_dates=from_dates,
    to_dates=to_dates,
    special_requirements=special_requirements
)

## **Data Exchange**

In [10]:
class DestinationDecodingSchema(pg.Object):
    themes: List[str]
    topics: List[str]
    behind_the_scenes: List[str]
    attractions: List[str]

class ThemeSynthesisSchema(pg.Object):
    behind_the_scenes: List[str]
    theme: List[str] 
    attractions: List[str]

class Itinerary(pg.Object):
    itinerary: str

class ItineraryEval(pg.Object):
    itinerary: str
    score: int #1-10

## **Agents**

In [74]:
## Finish the Job
class Done(lf.agentic.Action):
    def call(self, session, *, lm=None, **kwargs):
        return "Done"


    
## Destination Decoding Agent
class DestinationDecodingAgent(lf.agentic.Action):
    """
        An agent is used to decode the destination country, cities, and attractions based, including analyzing the country's cultural contradctions, 
        identifying seasonal exclusives, and mapping untapped overlaps between history, art, and outdoor adventures.
    """
    content: str

    def call(self, session, *, lm=lm_claude, **kwargs):

        prompt = """Your task is to decode the destination country, cities, and attractions based on the content: {{content}}
                    - Analyze the country’s cultural contradictions (e.g., Vietnam’s French colonial architecture vs. ancient Cham temples).
                    - Identify seasonal exclusives (e.g., December = fisherman’s net-mending rituals in Hoi An).
                    - Map untapped overlaps between history, art, and outdoor adventures (e.g., hiking to hill tribe villages known for indigo dyeing)."""
        
        result = session.query(prompt,
                               lm=lm,
                               content=self.content)
        return result
    


##Theme Synthesis Agent
class ThemeSynthesisAgent(lf.agentic.Action):
    """ 
        An agent is used to finalize the themes/topcis for the trip based on the current context, including creating a unique theme (ensure the theme ties together) 
        for the next step detailed trip plan .
    """
    content: str

    def call(self, session, *, lm=lm_claude, **kwargs):

        prompt = """Your task is to finalize the themes/topcis for the trip based on the current context {{content}}
                        - Create a unique theme based on the above.
                        - Ensure the theme ties together
                """
        
        result = session.query(prompt,
                               lm=lm, 
                               content=self.content
                               )
        return result
    


## Detailed D2D Planner Agent
class PlannerAgent(lf.agentic.Action):
    """ 
        An agent is used to finalize the itinerary based on all the content, including creating a detailed itinerary.
    """
    content: str

    allow_symbolic_assignment = True  

    def call(self, session: lf.agentic.Session, *, lm=lm_claude, **kwargs):
        prompt = """You task is to finalize the itinerary based on all the content: {{content}},that aligns with the themes/topics, and ensuring the itinerary is flexible to the family's interests and preferences. 
        Please create a detailed travel itinerary in markdown format with the following structure and make sure Google Map link is accurate:

        ## Introduction
        Begin with engaging storytelling about the trip (2-3 sentences) for 11 years old kid, including:
        - Background of the destination country
        - Overview of cities to be visited
        - Key attractions and significance
        - Purpose of the trip

        ## Trip Overview
        - **Dates**: [Start Date] to [End Date] ([X] days)
        - **Destinations**: List main cities/regions to be visited
        - **[Overall Route Map](Google Maps link with all major destinations)**

        ## Daily Itinerary

        ### **Day 1: 03/30 | [Locations]**
        Theme/Focus: [Theme/Focus] summarization for 11 years old kid
        [Day 1 Route Map](Google Maps link with all Day 1 locations)

        | Time | Activity | Location | Notes |
        |------|----------|----------|-------|
        | 09:00-10:30 | [Activity] | [Location] | [Important details] |
        | 11:00-13:00 | [Activity] | [Location] | [Important details] |
        | ... | ... | ... | ... |

        ### **Day 2: 03/31 | [Locations]**
        Theme/Focus: [Theme/Focus]
        [Day 2 Route Map](Google Maps link with all Day 2 locations)

        | Time | Activity | Location | Notes |
        |------|----------|----------|-------|
        | ... | ... | ... | ... |

        ## Other Tips
        - [General Tips]
        - [Transportation Tips]
        - [Food and Dining]
        - [Shopping and Local Products]
        - [Emergency Contacts]

        """

        result = session.query(prompt,
                               lm=lm,
                               content=self.content
                               )
        return result
    


class ShareTrip(lf.agentic.Action):
    """An agent is used to share the itinerary after it finished."""
    
    markdown_content: str

    allow_symbolic_assignment = True  

    def call(self, session, *, lm=lm_claude, token=notion_key, parent_id=notion_travel_database_id, **kwargs):
        create_page_action= NotionAction(
        operation="create_page",
        parent_id=parent_id,
        token=token,
        parent_type="page",
        markdown_content=self.markdown_content,
        debug_markdown=True  # Enable debugging
        # token will be taken from environment variable NOTION_API_TOKEN if not specified
    )
        result = create_page_action(session=session, lm=lm)

        if "error" not in result:
            print(result['success_message'])
            # This will print something like:
            # "Successfully created Notion page: 'My Trip to Hawaii'
            # View your page at: https://www.notion.so/abc123..."
        else:
            print(f"Error: {result['error']}")
        
        return result
    



class NextStep(pg.Object):
    step: int
    thoughts: list[str]
    action: DestinationDecodingAgent | ThemeSynthesisAgent | PlannerAgent | None | Done



class TripAgent(lf.agentic.Action):
    """The TripAgent is a agent that can plan the trip based on the family requests"""
    family_requests: FamilyRequests
    rounds: int

    allow_symbolic_assignment = True 

    def call(self, session, *, lm=lf.LanguageModel, **kwargs):
        past_steps = []
        final_plan = None
        
        prompt = """Your task is to help design the trip plan for the family, here is the family requests:{{family_requests}}, 
        you already have done these steps:{{past_steps}}, now please think step by step and come up with the next step"""
        
        for round in range(self.rounds):
            next_step = session.query(prompt,
                            NextStep, 
                            lm=lm,
                            family_requests=self.family_requests,
                            past_steps=past_steps,
                        )
    
            past_steps.append(next_step)
            lf.logging.info(str(next_step))
            next_action = next_step.action

            if next_action is not None and not isinstance(next_action, Done):    
                current_plan = next_action()
                past_steps.append(current_plan)

                    # Save the final plan from PlannerAgent to use for ShareTrip
                if isinstance(next_action, PlannerAgent):
                    final_plan = current_plan

            else:
                break

                    # After planning is complete, explicitly share the trip if we have a final plan
        if final_plan:

            share_action = ShareTrip(markdown_content=final_plan)
            share_result = share_action(session=session, lm=lm)
            return share_result
        return current_plan
    


## **Automation**

In [75]:
my_trip_agent = TripAgent(family_requests=family_requests, rounds=4)
result = my_trip_agent(lm=lm_claude)

Html(content='<details open class="pyglove session"><summary><div class="summary-name">agent_session</div><div class="summary-title">Session(...)</div></summary><table class="tab-control"><tr><td><div class="tab-button-group top" id="control-5007498544-button-group"><button class="tab-button selected" onclick="openTab(event, \'control-5007498544\', \'control-5007498544-0\')"><span class="label"><span class="execution-tab-title">execution<div class="label-container"><span class="label execution-time not-started badge" id="control-5007012208">(Not started)</span><span class="tooltip" id="control-5007195984">Execution not started.</span></div><div class="label-container"><span class="label usage-summary badge" id="control-5007035328">0.000</span><span class="tooltip" id="control-5007498928">UsageSummary(\n  cached = AggregatedUsage(\n    total = LMSamplingUsage(\n      prompt_tokens = 0,\n      completion_tokens = 0,\n      total_tokens = 0,\n      num_requests = 0,\n      estimated_cost = 0.000,\n      retry_stats = RetryStats(\n        num_occurences = 0,\n        total_wait_interval = 0.000,\n        total_call_interval = 0.000,\n        errors = {}\n      )\n    ),\n    breakdown = {}\n  ),\n  uncached = AggregatedUsage(\n    total = LMSamplingUsage(\n      prompt_tokens = 0,\n      completion_tokens = 0,\n      total_tokens = 0,\n      num_requests = 0,\n      estimated_cost = 0.000,\n      retry_stats = RetryStats(\n        num_occurences = 0,\n        total_wait_interval = 0.000,\n        total_call_interval = 0.000,\n        errors = {}\n      )\n    ),\n    breakdown = {}\n  )\n)</span></div></span></span></button></div></td></tr><tr><td><div class="tab-content-group top" id="control-5007498544-content-group"><div class="tab-content selected" id="control-5007498544-0"><table class="tab-control"><tr><td><div class="tab-button-group left" id="control-5007010480-button-group"></div></td><td><div class="tab-content-group left" id="control-5007010480-content-group"></div></td></tr></table></div></div></td></tr></table></details>', style_files=StyleFiles(), styles=Styles(parts={'\n        /* Summary styles. */\n        details.pyglove summary {\n          font-weight: bold;\n          margin: -0.5em -0.5em 0;\n          padding: 0.5em;\n        }\n        .summary-name {\n          display: inline;\n          padding: 3px 5px 3px 5px;\n          margin: 0 5px;\n          border-radius: 3px;\n        }\n        .summary-title {\n          display: inline;\n        }\n        .summary-name + div.summary-title {\n          display: inline;\n          color: #aaa;\n        }\n        .summary-title:hover + span.tooltip {\n          visibility: visible;\n        }\n        .summary-name:hover > span.tooltip {\n          visibility: visible;\n          background-color: darkblue;\n        }\n        ': 1, '\n        span.tooltip {\n          visibility: hidden;\n          white-space: pre-wrap;\n          font-weight: normal;\n          background-color: #484848;\n          color: #fff;\n          padding: 10px;\n          border-radius: 6px;\n          position: absolute;\n          z-index: 1;\n        }\n        span.tooltip:hover {\n          visibility: visible;\n        }\n        .tooltip.html-content {\n          white-space: inherit;\n          background-color: white;\n          color: inherit;\n          box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px;\n        }\n        ': 2, '\n        .label:hover + .tooltip {\n          visibility: visible;\n        }\n        ': 2, '\n        .label {\n          display: inline-block;\n          color: inherit;\n          padding: 5px;\n        }\n        .label-container {\n            display: inline-block;\n        }\n        ': 3, '\n        .badge {\n          background-color: #EEE;\n          border-radius: 5px;\n          color: #777;\n        }\n        ': 2, "\n        .usage-summary.label {\n            display: inline-flex;\n            border-radius: 5px;\n            paddi

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [85]:
with open("../../Apps/Data/final_plan.md", "r") as file:
    final_plan = file.read()
Markdown(final_plan)


# Netherlands & Belgium Family Adventure: Timeless Innovations

## Introduction
Imagine stepping into a world where windmills spin against colorful tulip fields, where medieval castles stand next to futuristic buildings, and where chocolate and cheese are national treasures! You're about to explore the Netherlands and Belgium, two small countries packed with amazing inventions both old and new. From Amsterdam's famous canals to Bruges' chocolate shops, you'll discover how people have been creating cool stuff here for hundreds of years - and you might even get to see some real knights' armor and space-age science along the way!

## Trip Overview
- **Dates**: March 30 to April 6, 2025 (8 days)
- **Destinations**: Amsterdam, Leiden, Delft, Bruges
- **[Overall Route Map](https://www.google.com/maps/dir/Amsterdam+Airport+Schiphol,+Evert+van+de+Beekstraat+202,+1118+CP+Schiphol,+Netherlands/Leiden,+Netherlands/Bruges,+Belgium/Delft,+Netherlands/Amsterdam+Airport+Schiphol,+Evert+van+de+Beekstraat+202,+1118+CP+Schiphol,+Netherlands/@51.9173867,3.7749293,9z/)**

## Daily Itinerary

### **Day 1: 03/30 | Amsterdam to Leiden**
Theme/Focus: Arrival & First Explorations
[Day 1 Route Map](https://www.google.com/maps/dir/Amsterdam+Airport+Schiphol,+Evert+van+de+Beekstraat+202,+1118+CP+Schiphol,+Netherlands/Leiden,+Netherlands/@52.2209857,4.4232499,11z/)

| Time | Activity | Location | Notes |
|------|----------|----------|-------|
| 09:00-10:30 | Arrival & Baggage Claim | Amsterdam Schiphol Airport | Collect luggage and get oriented |
| 10:30-11:30 | Train to Leiden | Schiphol to Leiden Central Station | Direct train, ~20 minutes |
| 11:30-13:00 | Check-in & Lunch | Accommodation in Leiden | Drop bags if room ready; otherwise store luggage |
| 13:00-15:00 | Leiden City Orientation Walk | Historic Center | See Leiden University (1575), Pieterskerk, and the canals |
| 15:00-17:00 | Leiden American Pilgrim Museum | Beschuitsteeg 9 | Learn about the Pilgrims who lived in Leiden before sailing to America |
| 17:00-19:00 | Dinner | Local restaurant | Try Dutch pancakes or traditional stamppot |
| 19:00-20:00 | Evening Canal Stroll | Leiden Canals | Enjoy the peaceful evening atmosphere |

### **Day 2: 03/31 | Amsterdam Museum Day**
Theme/Focus: Art & Science Exploration
[Day 2 Route Map](https://www.google.com/maps/dir/Leiden,+Netherlands/Rijksmuseum,+Museumstraat,+Amsterdam,+Netherlands/NEMO+Science+Museum,+Oosterdok,+Amsterdam,+Netherlands/Leiden,+Netherlands/@52.1997719,4.4232499,11z/)

| Time | Activity | Location | Notes |
|------|----------|----------|-------|
| 08:00-09:00 | Breakfast | Leiden accommodation | |
| 09:00-10:00 | Train to Amsterdam | Leiden to Amsterdam Central | ~35 minute journey |
| 10:30-13:00 | Rijksmuseum | Museumstraat 1, Amsterdam | Pre-booked visit; see Rembrandt's Night Watch and Dutch Masters |
| 13:00-14:00 | Lunch | Museum café or nearby | |
| 14:30-17:00 | NEMO Science Museum | Oosterdok 2, Amsterdam | Interactive science exhibits perfect for your 11-year-old |
| 17:30-19:00 | Dinner | Amsterdam | Try Indonesian rijsttafel, a Dutch colonial heritage |
| 19:00-20:00 | Train back to Leiden | Amsterdam to Leiden | Evening return |

### **Day 3: 04/01 | Leiden Exploration**
Theme/Focus: Science History & Natural Wonders
[Day 3 Route Map](https://www.google.com/maps/dir/Leiden,+Netherlands/Hortus+botanicus+Leiden,+Rapenburg,+Leiden,+Netherlands/Naturalis+Biodiversity+Center,+Darwinweg,+Leiden,+Netherlands/Museum+Boerhaave,+Lange+Sint+Agnietenstraat,+Leiden,+Netherlands/@52.1599255,4.4750259,14z/)

| Time | Activity | Location | Notes |
|------|----------|----------|-------|
| 09:00-10:00 | Breakfast | Leiden accommodation | |
| 10:00-12:00 | Hortus Botanicus | Rapenburg 73 | One of the oldest botanical gardens in the world (1590) |
| 12:00-13:30 | Lunch | Near Naturalis | |
| 13:30-16:00 | Naturalis Biodiversity Center | Darwinweg 2 | Dinosaur skeletons and natural history exhibits |
| 16:30-18:00 | Museum Boerhaave | Lange St. Agnietenstraat 10 | Science history museum with interactive exhibits |
| 18:30-20:00 | Dinner | Leiden center | Try local seafood specialties |

### **Day 4: 04/02 | Leiden to Bruges**
Theme/Focus: Medieval Marvels
[Day 4 Route Map](https://www.google.com/maps/dir/Leiden,+Netherlands/Bruges,+Belgium/@51.5401003,3.5865177,9z/)

| Time | Activity | Location | Notes |
|------|----------|----------|-------|
| 08:00-09:00 | Breakfast & Check-out | Leiden accommodation | |
| 09:00-12:00 | Train to Bruges | Leiden to Bruges | ~3 hour journey with connection in Antwerp |
| 12:00-13:30 | Check-in & Lunch | Bruges accommodation | |
| 14:00-16:00 | Bruges Walking Tour | Historic Center | UNESCO World Heritage site with medieval architecture |
| 16:00-17:30 | Belfry Tower Climb | Market Square | 366 steps for panoramic views (optional) |
| 17:30-19:00 | Dinner | Bruges restaurant | Try Belgian specialties like waterzooi or carbonade flamande |
| 19:00-20:30 | Evening Boat Tour | Bruges Canals | See the "Venice of the North" from the water |

### **Day 5: 04/03 | Bruges to Delft**
Theme/Focus: Chocolate, History & Travel
[Day 5 Route Map](https://www.google.com/maps/dir/Bruges,+Belgium/Choco-Story+Brugge,+Wijnzakstraat,+Bruges,+Belgium/Historium+Brugge,+Markt,+Bruges,+Belgium/Delft,+Netherlands/@51.7688555,3.5865177,9z/)

| Time | Activity | Location | Notes |
|------|----------|----------|-------|
| 08:30-09:30 | Breakfast & Check-out | Bruges accommodation | |
| 09:30-11:00 | Choco-Story Museum | Wijnzakstraat 2 | Belgian chocolate history with tastings |
| 11:00-12:30 | Historium Bruges | Markt 1 | Interactive medieval history experience |
| 12:30-13:30 | Lunch | Market Square | |
| 13:30-17:00 | Train to Delft | Bruges to Delft | ~3 hour journey with connection in Rotterdam |
| 17:00-18:00 | Check-in | Delft accommodation | |
| 18:00-19:30 | Dinner & Evening Walk | Delft Market Square | See the historic center illuminated |

### **Day 6: 04/04 | Delft Exploration**
Theme/Focus: Dutch Innovation & Design
[Day 6 Route Map](https://www.google.com/maps/dir/Delft,+Netherlands/Royal+Delft+-+The+Delftware+Factory,+Rotterdamseweg,+Delft,+Netherlands/Science+Centre+Delft,+Mijnbouwstraat,+Delft,+Netherlands/Nieuwe+Kerk,+Markt,+Delft,+Netherlands/@52.0060711,4.3499701,15z/)

| Time | Activity | Location | Notes |
|------|----------|----------|-------|
| 09:00-10:00 | Breakfast | Delft accommodation | |
| 10:00-12:00 | Royal Delft Factory | Rotterdamseweg 196 | See how famous blue Delftware is made |
| 12:00-13:30 | Lunch | Near TU Delft | |
| 13:30-16:00 | Science Centre Delft | Mijnbouwstraat 120 | Hands-on science exhibits at TU Delft |
| 16:30-18:00 | Nieuwe Kerk Tower | Market Square | Climb the tower for views and see royal tombs |
| 18:30-20:00 | Dinner | Delft center | Try Dutch specialties |

### **Day 7: 04/05 | Day Trip to The Hague**
Theme/Focus: Knights, Art & Beach
[Day 7 Route Map](https://www.google.com/maps/dir/Delft,+Netherlands/Mauritshuis,+Plein,+The+Hague,+Netherlands/Louwman+Museum,+Leidsestraatweg,+The+Hague,+Netherlands/Scheveningen+Beach,+Scheveningen,+The+Hague,+Netherlands/Delft,+Netherlands/@52.0673591,4.2499701,12z/)

| Time | Activity | Location | Notes |
|------|----------|----------|-------|
| 08:30-09:30 | Breakfast | Delft accommodation | |
| 09:30-10:00 | Tram to The Hague | Delft to The Hague | ~15 minute journey |
| 10:00-12:00 | Mauritshuis | Plein 29, The Hague | See Vermeer's "Girl with a Pearl Earring" |
| 12:00-13:30 | Lunch | Near Mauritshuis | |
| 13:30-15:30 | Louwman Museum | Leidsestraatweg 57 | World's oldest private car collection |
| 16:00-18:00 | Scheveningen Beach | The Hague | Walk on the pier and enjoy the North Sea views |
| 18:00-19:30 | Dinner | Scheveningen | Seafood dinner with beach views |
| 19:30-20:30 | Return to Delft | The Hague to Delft | Evening tram back |

### **Day 8: 04/06 | Delft to Amsterdam Airport**
Theme/Focus: Final Explorations & Departure
[Day 8 Route Map](https://www.google.com/maps/dir/Delft,+Netherlands/Delft+City+Hall,+Markt,+Delft,+Netherlands/Amsterdam+Airport+Schiphol,+Evert+van+de+Beekstraat+202,+1118+CP+Schiphol,+Netherlands/@52.1209857,4.3499701,11z/)

| Time | Activity | Location | Notes |
|------|----------|----------|-------|
| 08:00-09:00 | Breakfast & Pack | Delft accommodation | |
| 09:00-10:00 | Check-out | Delft accommodation | Store luggage if needed |
| 10:00-11:30 | Delft City Hall & Market | Market Square | Final souvenir shopping |
| 11:30-13:00 | Lunch | Delft center | Last Dutch meal |
| 13:00-14:30 | Train to Schiphol | Delft to Schiphol Airport | ~45 minute journey |
| 14:30-17:00 | Airport Check-in & Security | Schiphol Airport | Allow plenty of time for international departure |
| 17:00+ | Departure | Schiphol Airport | |

## Other Tips

- **Weather Preparation**: Pack layers for spring weather (40-60°F/5-15°C) with possible rain showers.
- **Transportation**: 
  - Consider purchasing an OV-chipkaart for public transportation in the Netherlands.
  - Trains between cities are reliable and frequent.
  - Bikes are everywhere in the Netherlands - watch for cyclists when crossing streets!

- **Food and Dining**:
  - Dutch specialties: stroopwafels, poffertjes (mini pancakes), herring, and cheese.
  - Belgian must-tries: waffles, frites with mayonnaise, and of course, chocolate.
  - Most restaurants have English menus and kid-friendly options.

- **Shopping and Local Products**:
  - Delftware (blue and white pottery) from Delft
  - Cheese from the Netherlands
  - Chocolate from Belgium
  - Tulip bulbs (if properly packaged for import)

- **Emergency Contacts**:
  - Emergency number in both countries: 112
  - U.S. Embassy in Netherlands: +31 70 310 2209
  - U.S. Embassy in Belgium: +32 2 811 4000